<a href="https://colab.research.google.com/github/mohammadreza-mohammadi94/PySpark-Analytics-Hub/blob/main/Delayed-Flights-PySpark-ML/DelayedFlightsPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Source](https://www.kaggle.com/datasets/patrickzel/flight-delay-and-cancellation-dataset-2019-2023)

# 1. Download Dataset

In [ ]:
!kaggle datasets download patrickzel/flight-delay-and-cancellation-dataset-2019-2023
!unzip /content/flight-delay-and-cancellation-dataset-2019-2023.zip

Dataset URL: https://www.kaggle.com/datasets/patrickzel/flight-delay-and-cancellation-dataset-2019-2023
License(s): other
 94% 131M/140M [00:02<00:00, 97.7MB/s]
100% 140M/140M [00:02<00:00, 50.4MB/s]
Archive:  /content/flight-delay-and-cancellation-dataset-2019-2023.zip
  inflating: dictionary.html         
  inflating: flights_sample_3m.csv   


# 2. Import Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import *
from pyspark.ml.feature import (VectorAssembler,
                                StringIndexer,
                                StandardScaler,
                                OneHotEncoder)
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# 3. Initializing Spark Session & Loading Dataset

In [ ]:
# Starting Spark Session
spark = SparkSession.builder \
    .appName("FlightDelay") \
    .config('spark.driver.memory', '12g') \
    .config('spark.sql.shuffle.partitions', '8') \
    .config('spark.default.parallelism', '8') \
    .getOrCreate()

## 3.1 Loading Dataset

In [ ]:
%%time

# Load
flights_df = (
    spark.read.csv(
        path="/content/flights_sample_3m.csv",
        header=True,
        inferSchema=True
    )
)

flights_df.cache()
flights_df.count()

CPU times: user 365 ms, sys: 37.5 ms, total: 402 ms
Wall time: 1min 24s


3000000

# 4. Analyzing & Preprocessing

## 4.1 Variable's Description

### 📆 **Date and Time Variables**
| **Variable**       | **Type**   | **Description**                                    |
|---------------------|-----------|----------------------------------------------------|
| **YEAR**           | Integer   | Year of the flight (e.g., 2019, 2020).             |
| **MONTH**          | Integer   | Month of the flight (1 = January, 12 = December).  |
| **DAY**            | Integer   | Day of the month.                                  |
| **DAY_OF_WEEK**    | Integer   | Day of the week (1 = Monday, 7 = Sunday).          |


### 🛫 **Flight Identification**
| **Variable**       | **Type**   | **Description**                                    |
|---------------------|-----------|----------------------------------------------------|
| **AIRLINE**        | String    | Airline code (e.g., AA for American Airlines).     |
| **FLIGHT_NUMBER**  | String    | Unique flight number assigned by the airline.     |
| **TAIL_NUMBER**    | String    | Aircraft registration number.                     |


### 🗺️ **Flight Route Information**
| **Variable**           | **Type**   | **Description**                                    |
|-------------------------|-----------|----------------------------------------------------|
| **ORIGIN_AIRPORT**      | String    | Origin airport code (e.g., JFK, LAX).             |
| **DESTINATION_AIRPORT** | String    | Destination airport code.                        |


### ⏱️ **Departure Details**
| **Variable**            | **Type**   | **Description**                                    |
|--------------------------|-----------|----------------------------------------------------|
| **SCHEDULED_DEPARTURE** | String    | Scheduled departure time (e.g., 14:30).           |
| **DEPARTURE_TIME**      | String    | Actual departure time.                            |
| **DEPARTURE_DELAY**     | Double    | Delay in minutes at departure. Positive if late.  |
| **TAXI_OUT**            | Double    | Time in minutes spent taxiing before takeoff.     |
| **WHEELS_OFF**          | String    | Time when the wheels left the ground.             |


### 🛬 **Arrival Details**
| **Variable**           | **Type**   | **Description**                                    |
|-------------------------|-----------|----------------------------------------------------|
| **SCHEDULED_ARRIVAL**  | String    | Scheduled arrival time.                           |
| **ARRIVAL_TIME**       | String    | Actual arrival time.                              |
| **ARRIVAL_DELAY**      | Double    | Delay in minutes at arrival. Positive if late.    |
| **TAXI_IN**            | Double    | Time in minutes spent taxiing after landing.      |
| **WHEELS_ON**          | String    | Time when the wheels touched the ground.          |


### 📊 **Flight Duration and Distance**
| **Variable**        | **Type**   | **Description**                                    |
|----------------------|-----------|----------------------------------------------------|
| **SCHEDULED_TIME**  | Double    | Planned duration of the flight (in minutes).       |
| **ELAPSED_TIME**    | Double    | Actual flight duration (in minutes).              |
| **AIR_TIME**        | Double    | Time spent in the air (in minutes).               |
| **DISTANCE**        | Double    | Distance traveled (in miles).                     |


### 🚦 **Flight Status**
| **Variable**        | **Type**   | **Description**                                    |
|----------------------|-----------|----------------------------------------------------|
| **DIVERTED**        | Integer   | 1 if the flight was diverted, 0 otherwise.         |
| **CANCELLED**       | Integer   | 1 if the flight was cancelled, 0 otherwise.        |
| **CANCELLATION_REASON** | String | Reason for cancellation: A (Airline), B (Weather), C (NAS), D (Security). |


### 🕒 **Delay Causes**
| **Variable**            | **Type**   | **Description**                                    |
|--------------------------|-----------|----------------------------------------------------|
| **AIR_SYSTEM_DELAY**    | Double    | Delay due to air traffic control system issues.   |
| **SECURITY_DELAY**      | Double    | Delay caused by security measures.               |
| **AIRLINE_DELAY**       | Double    | Delay caused by airline operations or crew.      |
| **LATE_AIRCRAFT_DELAY** | Double    | Delay caused by the late arrival of the aircraft. |
| **WEATHER_DELAY**       | Double    | Delay caused by weather conditions.              |



## 4.2 Dataset Structure

In [ ]:
print(" Dimension:\n", flights_df.count(), len(flights_df.columns))

 Dimension:
 3000000 32


In [ ]:
flights_df.show(5)

+----------+--------------------+--------------------+------------+--------+---------+------+-------------------+----+--------------------+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+------------+--------+--------+-----------------+-----------------+-------------+------------------+-----------------------+
|   FL_DATE|             AIRLINE|         AIRLINE_DOT|AIRLINE_CODE|DOT_CODE|FL_NUMBER|ORIGIN|        ORIGIN_CITY|DEST|           DEST_CITY|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|DELAY_DUE_CARRIER|DELAY_DUE_WEATHER|DELAY_DUE_NAS|DELAY_DUE_SECURITY|DELAY_DUE_LATE_AIRCRAFT|
+----------+--------------------+--------------------+------------+--------+---------+------+-------------------+----+--------------------+------------+--------

In [ ]:
flights_df.printSchema()

root
 |-- FL_DATE: date (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- AIRLINE_DOT: string (nullable = true)
 |-- AIRLINE_CODE: string (nullable = true)
 |-- DOT_CODE: integer (nullable = true)
 |-- FL_NUMBER: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- ORIGIN_CITY: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEST_CITY: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_TIME: double (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- TAXI_OUT: double (nullable = true)
 |-- WHEELS_OFF: double (nullable = true)
 |-- WHEELS_ON: double (nullable = true)
 |-- TAXI_IN: double (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- ARR_TIME: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)

## 4.3 Missing Values

In [ ]:
def check_missing_values(df):
    print("Column's Missing Values: \n")
    for col in df.columns:
        nans = df.filter(f.col(col).isNull()).count()
        if nans > 0:
            print(f"{col}: {nans}")

check_missing_values(flights_df)

Column's Missing Values: 

DEP_TIME: 77615
DEP_DELAY: 77644
TAXI_OUT: 78806
WHEELS_OFF: 78806
WHEELS_ON: 79944
TAXI_IN: 79944
ARR_TIME: 79942
ARR_DELAY: 86198
CANCELLATION_CODE: 2920860
CRS_ELAPSED_TIME: 14
ELAPSED_TIME: 86198
AIR_TIME: 86198
DELAY_DUE_CARRIER: 2466137
DELAY_DUE_WEATHER: 2466137
DELAY_DUE_NAS: 2466137
DELAY_DUE_SECURITY: 2466137
DELAY_DUE_LATE_AIRCRAFT: 2466137


In [ ]:
flights_df.where("ARR_TIME > 0.0").show(10)

+----------+--------------------+--------------------+------------+--------+---------+------+-------------------+----+--------------------+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+------------+--------+--------+-----------------+-----------------+-------------+------------------+-----------------------+
|   FL_DATE|             AIRLINE|         AIRLINE_DOT|AIRLINE_CODE|DOT_CODE|FL_NUMBER|ORIGIN|        ORIGIN_CITY|DEST|           DEST_CITY|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|DELAY_DUE_CARRIER|DELAY_DUE_WEATHER|DELAY_DUE_NAS|DELAY_DUE_SECURITY|DELAY_DUE_LATE_AIRCRAFT|
+----------+--------------------+--------------------+------------+--------+---------+------+-------------------+----+--------------------+------------+--------

## 4.4 Statistical Overview

In [ ]:
%%time
flights_df.select("DEP_DELAY", "ARR_DELAY", "DISTANCE", "ELAPSED_TIME", "DELAY_DUE_LATE_AIRCRAFT",
                  "DELAY_DUE_SECURITY", "DELAY_DUE_CARRIER", "CANCELLED") \
      .summary("count", "mean", "stddev", "min", "max") \
      .show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------------+-------------------+------------------+-------------------+
|summary|         DEP_DELAY|        ARR_DELAY|         DISTANCE|     ELAPSED_TIME|DELAY_DUE_LATE_AIRCRAFT| DELAY_DUE_SECURITY| DELAY_DUE_CARRIER|          CANCELLED|
+-------+------------------+-----------------+-----------------+-----------------+-----------------------+-------------------+------------------+-------------------+
|  count|           2922356|          2913802|          3000000|          2913802|                 533863|             533863|            533863|            3000000|
|   mean|10.123326179288219|4.260858150279257|809.3615516666666|136.6205411349158|     25.471281958105358|0.14593069757596985|24.759086132584578|            0.02638|
| stddev| 49.25183487489521|51.17482436059588|587.8939382449503|71.67581550996236|     55.766892035226995| 3.5820528160419096| 71.77184461920068|0.16026260999175043|
|   

## 4.5 Creating Target Variable

In [ ]:
flights_df = (
    flights_df.withColumn(
    "Is_Delayed",
    f.when(f.col("DEP_DELAY") >= 5.0, 1.0).otherwise(0.0)
    )
)

In [ ]:
flights_df.groupBy("Is_Delayed").count().show()

+----------+-------+
|Is_Delayed|  count|
+----------+-------+
|       1.0| 794501|
|       0.0|2205499|
+----------+-------+



## 4.6 Feature Eng

In [ ]:
apt_traffic = flights_df.groupBy("ORIGIN").agg(
    f.count("*").alias("Daily_Flights")
    )

flights_df = flights_df.join(apt_traffic, "ORIGIN")

# Creating Is_Busy_Apt
flights_df = flights_df.withColumn("Is_Busy_Apt",
                                   f.when(f.col("Daily_Flights")> 1000, 1).otherwise(0))

In [ ]:
# Extract Month, Day, Year

flights_df = (
    flights_df.withColumn("Year", f.year(flights_df['FL_DATE']))
    .withColumn("Month", f.month(flights_df['FL_DATE']))
    .withColumn("Day", f.dayofmonth(flights_df['FL_DATE'])
    )
)

flights_df.show(2)

+------+----------+----------------+--------------------+------------+--------+---------+------------+----+---------------+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+------------+--------+--------+-----------------+-----------------+-------------+------------------+-----------------------+----------+-------------+-----------+----+-----+---+
|ORIGIN|   FL_DATE|         AIRLINE|         AIRLINE_DOT|AIRLINE_CODE|DOT_CODE|FL_NUMBER| ORIGIN_CITY|DEST|      DEST_CITY|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|DELAY_DUE_CARRIER|DELAY_DUE_WEATHER|DELAY_DUE_NAS|DELAY_DUE_SECURITY|DELAY_DUE_LATE_AIRCRAFT|Is_Delayed|Daily_Flights|Is_Busy_Apt|Year|Month|Day|
+------+----------+----------------+--------------------+------------+--------+---------

## 4.7 Selecting Relevant Features

In [ ]:
# flights_df.columns
flights_df.show(1)

+------+----------+----------------+--------------------+------------+--------+---------+-----------+----+---------------+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+------------+--------+--------+-----------------+-----------------+-------------+------------------+-----------------------+----------+-------------+-----------+----+-----+---+
|ORIGIN|   FL_DATE|         AIRLINE|         AIRLINE_DOT|AIRLINE_CODE|DOT_CODE|FL_NUMBER|ORIGIN_CITY|DEST|      DEST_CITY|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|DELAY_DUE_CARRIER|DELAY_DUE_WEATHER|DELAY_DUE_NAS|DELAY_DUE_SECURITY|DELAY_DUE_LATE_AIRCRAFT|Is_Delayed|Daily_Flights|Is_Busy_Apt|Year|Month|Day|
+------+----------+----------------+--------------------+------------+--------+---------+-

In [ ]:
df = flights_df.select("Month", "Day", "DEP_TIME", "DEP_DELAY",
                       "AIRLINE_CODE", "Is_Busy_Apt",
                       "Is_Delayed")

In [ ]:
# print(df.columns)
df.show()

+-----+---+--------+---------+------------+-----------+----------+
|Month|Day|DEP_TIME|DEP_DELAY|AIRLINE_CODE|Is_Busy_Apt|Is_Delayed|
+-----+---+--------+---------+------------+-----------+----------+
|    2| 12|   527.0|     -3.0|          NK|          1|       0.0|
|    5|  5|   800.0|     -3.0|          B6|          1|       0.0|
|   11| 12|   720.0|    -10.0|          DL|          1|       0.0|
|   12| 28|  1835.0|     30.0|          OO|          1|       1.0|
|    7| 18|  2254.0|     79.0|          WN|          1|       1.0|
|    7|  3|  1350.0|    -10.0|          B6|          1|       0.0|
|    4| 16|  1409.0|     -5.0|          MQ|          1|       0.0|
|    2| 26|  1538.0|     -6.0|          B6|          1|       0.0|
|    4|  2|  1310.0|     -9.0|          F9|          1|       0.0|
|    3| 29|  1955.0|     -5.0|          UA|          1|       0.0|
|    7|  9|  1921.0|    111.0|          B6|          1|       1.0|
|   10| 13|   637.0|      2.0|          WN|          1|       

# 5. Vectorization & Encoding

In [ ]:
# Categorical and numerical variables
categorical_variables = ["AIRLINE_CODE"]
numerical_variables = ["Month", "Day", "Is_Busy_Apt", "AIRLINE_CODE_INDEXED"]
target_variable = "Is_Delayed"

In [ ]:
# Convert categorical varibale to numeric
indexer = StringIndexer(
    inputCol="AIRLINE_CODE",
    outputCol="AIRLINE_CODE_INDEXED"
)

df = indexer.fit(df).transform(df)

In [ ]:
df.show(1) # Check df

+-----+---+--------+---------+------------+-----------+----------+--------------------+
|Month|Day|DEP_TIME|DEP_DELAY|AIRLINE_CODE|Is_Busy_Apt|Is_Delayed|AIRLINE_CODE_INDEXED|
+-----+---+--------+---------+------------+-----------+----------+--------------------+
|    2| 12|   527.0|     -3.0|          NK|          1|       0.0|                11.0|
+-----+---+--------+---------+------------+-----------+----------+--------------------+
only showing top 1 row



In [ ]:
# Feature's vectorization
vectorizer = VectorAssembler(
    inputCols= numerical_variables,
    outputCol="features_vector"
)

data = vectorizer.transform(df)

In [ ]:
# Define random forest classifier model
rf = RandomForestClassifier(
    featuresCol="features_vector",
    labelCol="Is_Delayed",
    numTrees=200,
    maxDepth=10
)

In [ ]:
data.show() # Check data

+-----+---+--------+---------+------------+-----------+----------+--------------------+-------------------+
|Month|Day|DEP_TIME|DEP_DELAY|AIRLINE_CODE|Is_Busy_Apt|Is_Delayed|AIRLINE_CODE_INDEXED|    features_vector|
+-----+---+--------+---------+------------+-----------+----------+--------------------+-------------------+
|    2| 12|   527.0|     -3.0|          NK|          1|       0.0|                11.0|[2.0,12.0,1.0,11.0]|
|    5|  5|   800.0|     -3.0|          B6|          1|       0.0|                 7.0|  [5.0,5.0,1.0,7.0]|
|   11| 12|   720.0|    -10.0|          DL|          1|       0.0|                 1.0|[11.0,12.0,1.0,1.0]|
|   12| 28|  1835.0|     30.0|          OO|          1|       1.0|                 3.0|[12.0,28.0,1.0,3.0]|
|    7| 18|  2254.0|     79.0|          WN|          1|       1.0|                 0.0| [7.0,18.0,1.0,0.0]|
|    7|  3|  1350.0|    -10.0|          B6|          1|       0.0|                 7.0|  [7.0,3.0,1.0,7.0]|
|    4| 16|  1409.0|     -5.

# 6. Training & Testing

In [ ]:
# Splitting Train/Test
train, test = data.randomSplit([0.8, 0.2], seed=42)

# Training
model = rf.fit(train)

In [ ]:
# Prediction
predictions = model.transform(test)
# Check predictions
predictions.show()

## 6.1 Confusion Matrix

In [ ]:
# Calculate additional metrics
tp = predictions.filter((f.col("prediction") == 1.0) & (f.col("Is_Delayed") == 1.0)).count()
fp = predictions.filter((f.col("prediction") == 1.0) & (f.col("Is_Delayed") == 0.0)).count()
fn = predictions.filter((f.col("prediction") == 0.0) & (f.col("Is_Delayed") == 1.0)).count()

# Avoid division by zero
precision = tp / (tp + fp) if (tp + fp) != 0 else 0
recall = tp / (tp + fn) if (tp + fn) != 0 else 0

# Print the results
print(f"Precision: {precision}")
print(f"Recall: {recall}")

## 6.2 Evaluation

In [ ]:
# Evaluate model
evaluator = BinaryClassificationEvaluator(
    labelCol="Is_Delayed",
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)

accuracy = evaluator.evaluate(predictions)

In [ ]:
accuracy

In [ ]:
rf = RandomForestClassifier(labelCol="Is_Delayed", featuresCol="features_vector")

# Define parameter grid
paramGrid = (ParamGridBuilder()
             .addGrid(rf.numTrees, [50, 100, 200])  # تعداد درخت‌ها
             .addGrid(rf.maxDepth, [5, 10, 20])  # عمق درخت‌ها
             .addGrid(rf.maxBins, [32, 64, 128])  # تعداد سطل‌ها
             .build())

# Define ClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol="Is_Delayed")

crossval = CrossValidator(estimator=rf,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

In [ ]:
# Train Model
cvModel = crossval.fit(train)

# Get prediction
predictions = cvModel.transform(test)

# Evaluation
accuracy = evaluator.evaluate(predictions)
print(f"Model Accuracy: {accuracy}")

In [ ]:
train.show()